In [1]:
!pip install transformers seqeval keras torch matplotlib seaborn tensorflow Keras-Preprocessing

   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   ------------------- -------------------- 20.5/42.6 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 42.6/42.6 kB 414.6 kB/s eta 0:00:00


In [23]:
sentences, labels = [], []
tag_values = set() # set of the values found 
with open('aij-wikiner-ru-wp3', encoding="utf8") as df:
    for line in df:
        sent_w = [] # for word
        # Tags correspons to this https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)
        sent_t = [] # for tag
        sent = line.split()
        for s in sent:
            tag = s.split('|')
            # getting rid of the word types, collecting only word and tag, putting them into separate groups
            sent_w.append(tag[0])
            sent_t.append(tag[2])
            if len(tag[2]) > 0:
                tag_values.update([tag[2]])

        # Basically this if statment is to check when we have empty line
        if len(sent_w) > 0:
            sentences.append(sent_w)
            labels.append(sent_t)

import random
tag_values = list(tag_values)
#random.shuffle(tag_values)
tag_values.append('PAD')
tag_values.append('X')
print(tag_values)


['B-MISC', 'O', 'I-PER', 'B-ORG', 'I-ORG', 'I-LOC', 'B-PER', 'B-LOC', 'I-MISC', 'PAD', 'X']


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

In [6]:
model = AutoModelForTokenClassification.from_pretrained('C:/source/other/NER_BERT_RU/model')
tokenizer = AutoTokenizer.from_pretrained('C:/source/other/NER_BERT_RU/model')

In [61]:
import torch
test_sentence = """Спустя несколько часов, ранним утром по Москве, информагентства сообщили, что в резиденции «Кымсусан» начались переговоры, ради которых визит и был организован. С российской стороны в состав делегации вошли, в частности, министры обороны и иностранных дел Андрей Белоусов и Сергей Лавров, а также главы РЖД и Роскосмоса.

"""
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cpu()


In [51]:
tag_values = ['O', 'I-LOC', 'B-MISC', 'B-LOC', 'B-PER', 'I-PER', 'I-MISC', 'B-ORG', 'I-ORG', 'PAD', 'X']

In [62]:
import numpy as np
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

# объединяем токены и метки
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

O	[CLS]
O	Спустя
O	несколько
O	часов
O	,
O	ранним
O	утром
O	по
I-LOC	Москве
O	,
O	информагентства
O	сообщили
O	,
O	что
O	в
O	резиденции
O	«
I-MISC	Кымсусан
O	»
O	начались
O	переговоры
O	,
O	ради
O	которых
O	визит
O	и
O	был
O	организован
X	.
O	С
O	российской
O	стороны
O	в
O	состав
O	делегации
O	вошли
O	,
O	в
O	частности
O	,
O	министры
O	обороны
O	и
O	иностранных
O	дел
I-PER	Андрей
I-PER	Белоусов
O	и
I-PER	Сергей
I-PER	Лавров
O	,
O	а
O	также
O	главы
I-ORG	РЖД
O	и
I-ORG	Роскосмоса
O	.
O	[SEP]
